<a href="https://colab.research.google.com/github/SUDHARSSHINI/Sample/blob/main/LSTM_Stock_Market_analysis_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# List of file paths for the five datasets
file_paths = [
    "/content/CBOE 10Y Bond 2008_2023.csv",
    "/content/Crude 2008_2023.csv",
    "/content/Gold 2008_2023.csv",
    "/content/NIFTY 2008_2023.csv",
    "/content/USDINR 2008_2023.csv"
]

# Define function to preprocess data
def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data

# Define function to prepare data for LSTM
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps, 0])
        y.append(data[i + n_steps, 0])
    return np.array(X), np.array(y)

# Define LSTM model architecture
def build_model(n_steps):
    model = Sequential([
        LSTM(units=50, activation='relu', input_shape=(n_steps, 1)),
        Dense(units=1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Iterate over each dataset
for file_path in file_paths:
    # Load dataset
    df = pd.read_csv(file_path)

    # Preprocess data
    if 'Close' not in df.columns:
        print("Close column not found in", file_path)
        continue

    data = df['Close'].values.reshape(-1, 1)
    scaled_data = preprocess_data(data)

    # Prepare data for LSTM
    n_steps = 30  # Number of time steps
    X, y = prepare_data(scaled_data, n_steps)
    X = X.reshape((X.shape[0], X.shape[1], 1))

    # Split data into training and testing sets
    split_ratio = 0.8
    split_index = int(split_ratio * len(X))
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Build and train LSTM model
    model = build_model(n_steps)
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    # Evaluate model
    mse = model.evaluate(X_test, y_test, verbose=0)
    print("Mean Squared Error for", file_path, ":", mse)

    # Make predictions
    predictions = model.predict(X_test)

    # Perform necessary inverse transformations and analysis
    # (e.g., inverse scaling to get actual price values, visualization)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Mean Squared Error for /content/CBOE 10Y Bond 2008_2023.csv : nan


31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Close column not found in /content/Crude 2008_2023.csv
Close column not found in /content/Gold 2008_2023.csv


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Mean Squared Error for /content/NIFTY 2008_2023.csv : nan
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Mean Squared Error for /content/USDINR 2008_2023.csv : nan
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Function to preprocess data
def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

# Function to prepare data for LSTM
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps, 0])
        y.append(data[i + n_steps, 0])
    return np.array(X), np.array(y)

# Function to build LSTM model
def build_model(n_steps):
    model = Sequential([
        LSTM(units=50, activation='relu', input_shape=(n_steps, 1)),
        Dense(units=1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Function to predict next day's trend
def predict_next_day_trend(model, scaler, data, n_steps):
    # Prepare data for predicting next day's trend
    last_30_days = data[-n_steps:].reshape(-1, 1)  # Reshape to have only one feature
    scaled_last_30_days = scaler.transform(last_30_days)
    X_next_day = scaled_last_30_days.reshape((1, n_steps, 1))

    # Predict next day's closing price
    next_day_pred = model.predict(X_next_day)

    # Inverse transform to get actual price value
    next_day_pred_actual = scaler.inverse_transform(next_day_pred)

    # Compare with today's closing price to determine trend
    today_close = data[-1][0]
    next_day_close_pred = next_day_pred_actual[0][0]
    if next_day_close_pred > today_close:
        return "upward"
    elif next_day_close_pred < today_close:
        return "downward"
    else:
        return "neutral or unchanged"

# List of file paths for the five datasets
file_paths = [
    "/content/CBOE 10Y Bond 2008_2023.csv",
    "/content/Crude 2008_2023.csv",
    "/content/Gold 2008_2023.csv",
    "/content/NIFTY 2008_2023.csv",
    "/content/USDINR 2008_2023.csv"
]

# Iterate over each dataset
for file_path in file_paths:
    # Load dataset
    df = pd.read_csv(file_path)

    # Preprocess data
    if 'Close' not in df.columns:
        print(f"Close column not found in {file_path}")
    else:
        data = df['Close'].values.reshape(-1, 1)
        scaled_data, scaler = preprocess_data(data)

        # Prepare data for LSTM
        n_steps = 30  # Number of time steps
        X, y = prepare_data(scaled_data, n_steps)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        # Build and train LSTM model
        model = build_model(n_steps)
        model.fit(X, y, epochs=50, batch_size=32, verbose=0)

        # Predict next day's trend
        next_day_trend = predict_next_day_trend(model, scaler, data, n_steps)
        print(f"For dataset {file_path}, the predicted trend for the next day is {next_day_trend}.")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
For dataset /content/CBOE 10Y Bond 2008_2023.csv, the predicted trend for the next day is neutral or unchanged.
Close column not found in /content/Crude 2008_2023.csv
Close column not found in /content/Gold 2008_2023.csv


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
For dataset /content/NIFTY 2008_2023.csv, the predicted trend for the next day is neutral or unchanged.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
For dataset /content/USDINR 2008_2023.csv, the predicted trend for the next day is neutral or unchanged.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Load all five datasets
file_paths = ['/content/CBOE 10Y Bond 2008_2023.csv', '/content/Crude 2008_2023.csv', '/content/Gold 2008_2023.csv', '/content/NIFTY 2008_2023.csv', '/content/USDINR 2008_2023.csv']
dfs = [pd.read_csv(file) for file in file_paths]
combined_df = pd.concat(dfs, ignore_index=True)

# Preprocessing
combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']] = \
    combined_df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].apply(pd.to_numeric, errors='coerce')

# Identify non-numeric values
non_numeric_cols = combined_df.select_dtypes(exclude=['number']).columns

# Handle non-numeric values
if non_numeric_cols.any():
    combined_df.drop(non_numeric_cols, axis=1, inplace=True)  # Drop non-numeric columns

# Impute missing values
combined_df.fillna(method='ffill', inplace=True)  # Forward fill missing values

# Normalize the data
scaler = MinMaxScaler()
y = combined_df['Close'].values  # Target variable y
X = combined_df.drop('Close', axis=1).values  # Features X

# Reshape features for LSTM input (samples, timesteps, features)
n_features = X.shape[1]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Normalize the target variable y
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1))  # Fit scaler only to the target variable y

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, n_features)))
model.add(Dense(1))  # Output layer
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)

# Inverse transform to get actual values
y_test_inv = scaler.inverse_transform(y_test)  # No need to reshape y_test
y_pred_inv = scaler.inverse_transform(y_pred)  # No need to reshape y_pred

mse = mean_squared_error(y_test_inv, y_pred_inv)
print("Mean Squared Error:", mse)

# Analyze results (optional)
# For example, plot actual vs. predicted values
plt.figure(figsize=(10, 6))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.xlabel('Time')
plt.ylabel('Close Price')
plt.title('Actual vs. Predicted Close Prices')
plt.legend()
plt.show()



ValueError: time data "Dec 29, 2023" doesn't match format "%Y-%m-%d", at position 4863. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
# Build LSTM model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(1, n_features), return_sequences=True))  # Increase LSTM units
model.add(LSTM(50, activation='relu'))  # Additional LSTM layer
model.add(Dense(1))  # Output layer
model.compile(optimizer='adam', loss='mse')

# Train the model for more epochs
history = model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = model.predict(X_test)

# Inverse transform to get actual values
y_test_inv = scaler.inverse_transform(y_test)  # No need to reshape y_test
y_pred_inv = scaler.inverse_transform(y_pred)  # No need to reshape y_pred

mse = mean_squared_error(y_test_inv, y_pred_inv)
print("Mean Squared Error:", mse)

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


NameError: name 'n_features' is not defined

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np

# Sample data (replace this with your actual data)
data = np.random.rand(100, 1)

# Normalize the data
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

# Split data into training and testing sets
train_size = int(len(data_normalized) * 0.8)
test_size = len(data_normalized) - train_size
train, test = data_normalized[0:train_size,:], data_normalized[train_size:len(data_normalized),:]

# Convert data to sequences
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        X.append(dataset[i:(i+look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 3
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build LSTM model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(1, look_back)))
model.add(Dropout(0.2))  # Adding dropout for regularization
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
mse = mean_squared_error(y_test_inv, y_pred_inv)
print("Mean Squared Error:", mse)





Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 139ms/step - loss: 0.2661 - val_loss: 0.3099
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.2503 - val_loss: 0.2901
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.2177 - val_loss: 0.2708
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2129 - val_loss: 0.2518
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1825 - val_loss: 0.2335
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1925 - val_loss: 0.2154
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1610 - val_loss: 0.1981
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1492 - val_loss: 0.1811
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.1506 - val_loss: 0.1647
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1483 - val_loss: 0.1495
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1383 - val_loss: 0.1357
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1227 - val_loss: 0.1234


In [ ]:
# Determine the trend based on predicted values and actual values
def find_trend(y_actual, y_predicted):
    trend = []
    for i in range(len(y_actual) - 1):
        if y_predicted[i + 1] > y_predicted[i]:
            trend.append('Upward')
        elif y_predicted[i + 1] < y_predicted[i]:
            trend.append('Downward')
        else:
            trend.append('No Change')
    return trend

# Find the trend
trend = find_trend(y_test_inv[:-1], y_pred_inv[:-1])

# Print the trend
print("Trend:", trend)


Trend: ['Upward', 'Upward', 'Upward', 'Upward', 'Downward', 'Downward', 'Downward', 'Upward', 'Upward', 'Downward', 'Downward', 'Downward', 'Upward', 'Upward']


In [ ]:
import pandas as pd

# Sample data
data = {
    'Date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04'],
    'Open': [100.0, 102.0, 105.0, 103.0],
    'High': [105.0, 103.5, 108.0, 104.5],
    'Low': [98.0, 100.5, 103.0, 99.0],
    'Close': [102.5, 100.8, 106.0, 102.2],
    'Adj Close': [101.0, 102.0, 104.5, 103.0],
    'Volume': [1000000, 1200000, 900000, 1100000]
}

# Convert data to DataFrame
df = pd.DataFrame(data)

# Calculate daily trend based on closing prices
df['Trend'] = df['Close'].diff().apply(lambda x: 'Upward' if x > 0 else 'Downward')

# Display the DataFrame with trend
print(df[['Date', 'Close', 'Trend']])


         Date  Close     Trend
0  2023-01-01  102.5  Downward
1  2023-01-02  100.8  Downward
2  2023-01-03  106.0    Upward
3  2023-01-04  102.2  Downward


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import numpy as np

# Sample data (replace this with your actual data)
data = {
    'Date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04'],
    'Open': [100.0, 102.0, 105.0, 103.0],
    'High': [105.0, 103.5, 108.0, 104.5],
    'Low': [98.0, 100.5, 103.0, 99.0],
    'Close': [102.5, 100.8, 106.0, 102.2],
    'Adj Close': [101.0, 102.0, 104.5, 103.0],
    'Volume': [1000000, 1200000, 900000, 1100000]
}

# Convert sample data to NumPy array
data_array = np.array(data['Close']).reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data_array)

# Prepare the data for LSTM model
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back):
        X.append(dataset[i:(i+look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 1
X, y = create_dataset(data_normalized, look_back)

# Reshape input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

# Build LSTM model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(1, look_back)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=100, batch_size=1, verbose=1)

# Predict the trend for the next day
next_day_data_normalized = data_normalized[-1].reshape(1, 1, 1)
predicted_normalized = model.predict(next_day_data_normalized)
predicted_value = scaler.inverse_transform(predicted_normalized)[0][0]

# Compare with the last actual value to determine trend
last_actual_value = data_array[-1][0]
if predicted_value > last_actual_value:
    trend = 'Upward'
elif predicted_value < last_actual_value:
    trend = 'Downward'
else:
    trend = 'No Change'

print("Predicted Value for Next Day:", predicted_value)
print("Last Actual Value:", last_actual_value)
print("Trend for Next Day:", trend)



Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.5513
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5562 
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2966     
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2933     
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1760     
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5311 
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5302 
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5232 
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1765 
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1646 
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5118 
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2840 
Epoch 13/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1603 
Epoch 14/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1699 
Epoch 15/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2655 
Epoch 16/100
3/3 ━━━

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import numpy as np

# Sample data (replace this with your actual data)
data = {
    'Date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04'],
    'Open': [50.0, 102.0, 50.0, 50.0],
    'High': [50.0, 95.5, 100.0, 104.5],
    'Low': [50.0, 100.5, 50.0, 99.0],
    'Close': [105.5, 100.8, 50.0, 102.2],
    'Adj Close': [101.0, 102.0, 50.5, 103.0],
    'Volume': [1000000, 1200000, 900000, 1100000]
}

# Convert sample data to NumPy array
data_array = np.array(data['Close']).reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data_array)

# Prepare the data for LSTM model
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back):
        X.append(dataset[i:(i+look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 1
X, y = create_dataset(data_normalized, look_back)

# Reshape input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

# Build LSTM model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(1, look_back)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=100, batch_size=1, verbose=1)

# Predict the trend for the next day
next_day_data_normalized = data_normalized[-1].reshape(1, 1, 1)
predicted_normalized = model.predict(next_day_data_normalized)
predicted_value = scaler.inverse_transform(predicted_normalized)[0][0]

# Compare with the last actual value to determine trend
last_actual_value = data_array[-1][0]
if predicted_value > last_actual_value:
    trend = 'Upward'
elif predicted_value < last_actual_value:
    trend = 'Downward'
else:
    trend = 'No Change'

print("Predicted Value for Next Day:", predicted_value)
print("Last Actual Value:", last_actual_value)
print("Trend for Next Day:", trend)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3958   
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3951     
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3859     
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6009 
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3745     
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6988 
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3739 
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5717 
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5740 
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6583 
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6372 
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6413 
Epoch 13/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3319 
Epoch 14/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3425 
Epoch 15/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6034 
Epoch 16/100
3/3 